For installation of **DSSEX** please follow [installation instructions](./readme.ipynb#Install-DSSEX).

Alternatively you can run ```%pip install dssex``` in a cell.

In [ ]:
import dssex
from egrid import create_objects, make_data_frames, check_frames, model_from_frames
import pandas as pd
pd.set_option('display.max_colwidth', None)

def print_frames(frames):
    """Prints pandas.DataFrame instances stored in a dict"""
    messages = frames.get('Message')
    if messages is not None:
        is_error = 1 < messages.level
        errors = messages.loc[is_error].message
        if not errors.empty:
            print('Errors\n')
            for _, error in errors.items():
                print()
                print(error)
            print()
    for key,df in frames.items():
        print()
        print(key)
        display(df)
        
def print_results(res):
    """Prints values returned from function dssex.estimate"""
    messages, vals = res
    if not messages.empty:
        print('\n\n')
        print('Messages')
        display(messages)
    last_index = len(vals) - 2
    for idx, df in enumerate(vals, -1):
        print('\n\n')
        print(f'{"RESULT >> " if idx==last_index else ""}Optimization Step: {idx}')
        for name in ['nodes', 'branches', 'injections']:
            print()
            print(name.title())
            display(df[name])    

# Estimation
## Scale Active Power P

In [ ]:
schema05 = """
       P=30
slack)-----------transformer_slack_00--(n00)--------line_00_01-------(n01)
                  y_lo=10k-10kj          |           y_lo=2k-2kj       |
                  y_tr=1µ+8µj            |           y_tr=2µ+2µj       |
                                         |                             |
                     _load00_0 <-------(n00)--|| cap00_1_            (n01)--> load01_0_
                       P10=12 Q10=7               Q10=-3                       P10=42
                       Exp_v_p=1                  Exp_v_q=2                    Q10=27
                       Exp_v_q=1
#.
Defk(id=kp min=0 max=1 )

# associate scaling factor 'kp' to active power P of injections 'load00_0' and 'load01_0'
Klink(id_of_injection(load00_0 load01_0) id_of_factor=kp part=p)
"""
frames05 = make_data_frames(create_objects(schema05))
print_frames(frames05)

In [ ]:
pd.DataFrame.from_records(check_frames(frames05), columns=['message','level'])

[step_params](./help_params.ipynb)

In [ ]:
model05 = model_from_frames(frames05)
# objective is 'meet active power measurement P'
step_params = [
    dict(objectives='P')
]
results05 = dssex.estimate(model05, step_params)

Results for two steps. 

- Initial power flow calculation
- Optimization

In [ ]:
print_results(results05)

## Scale Active Power P and Reactive Power Q

In [ ]:
schema06 = """
       P=30 Q=15
slack)-----------transformer_slack_00--(n00)--------line_00_01-------(n01)
                  y_lo=10k-10kj          |           y_lo=2k-2kj       |
                  y_tr=1µ+8µj            |           y_tr=2µ+2µj       |
                                         |                             |
                     _load00_0 <-------(n00)--|| cap00_1_            (n01)--> load01_0_
                       P10=12 Q10=7               Q10=-3                       P10=42
                       Exp_v_p=1                  Exp_v_q=2                    Q10=27
                       Exp_v_q=1
#.
Defk(id(kp kq) min=0 max=1)

# associate scaling factor 'kp'/'kq' to active/reactive power P/Q of injections 'load00_0' and 'load01_0'
Klink(id_of_injection(load00_0  load01_0) id_of_factor(kp kq) part(p q))
"""
frames06 = make_data_frames(create_objects(schema06))
print_frames(frames06)

In [ ]:
pd.DataFrame.from_records(check_frames(frames05), columns=['message','level'])

In [ ]:
model06 = model_from_frames(frames06)
# objective is 'meet active power measurements P and Q'
step_params = [
    dict(objectives='PQ')
]
results06 = dssex.estimate(model06, step_params)

In [ ]:
print_results(results06)